# Graphene FET I-V Characteristics

This notebook demonstrates parameter sweeps and plotting of I-V curves for the GFET model.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
import sys
sys.path.append('../src')

# For SPICE simulation (if available)
try:
    from spice_interface import GFETSimulator
    SPICE_AVAILABLE = True
except ImportError:
    SPICE_AVAILABLE = False
    print("SPICE interface not available, using analytical model")

plt.style.use('seaborn-v0_8-darkgrid')
%matplotlib inline

## Analytical GFET Model

Simplified analytical model for quick calculations.

In [ ]:
class AnalyticalGFET:
    """Analytical GFET model for quick I-V calculations"""
    
    def __init__(self, W=10e-6, L=1e-6, Cox=1.15e-8, mu=10000, Vdirac=0.0):
        self.W = W  # Channel width (m)
        self.L = L  # Channel length (m)
        self.Cox = Cox  # Gate oxide capacitance (F/cm^2)
        self.mu = mu  # Mobility (cm^2/V·s)
        self.Vdirac = Vdirac  # Dirac point voltage (V)
        self.q = 1.6e-19  # Elementary charge (C)
        
    def carrier_density(self, Vgs):
        """Calculate carrier density (cm^-2)"""
        n_gate = self.Cox * 1e4 * (Vgs - self.Vdirac) / self.q
        n_min = 1e10  # Residual carrier density
        return np.sqrt(n_gate**2 + n_min**2)
    
    def drain_current(self, Vgs, Vds):
        """Calculate drain current (A)"""
        n = self.carrier_density(Vgs)
        
        # Linear region
        if abs(Vds) < 0.1:
            alpha = self.q * self.mu * n * 1e-4 * (self.W / self.L)
            return alpha * Vds
        
        # Saturation region (simplified)
        vF = 1e6  # Fermi velocity (m/s)
        beta = vF * np.sqrt(np.pi * n * 1e4)  # Convert to m^-2
        Ids = self.W * self.q * n * 1e4 * beta * np.tanh(Vds / 0.5)
        return Ids

# Create GFET instance
gfet = AnalyticalGFET()

## Output Characteristics (Ids vs Vds)

In [ ]:
# Sweep parameters
Vds_range = np.linspace(0, 2.0, 100)
Vgs_values = np.linspace(-1.0, 1.0, 5)

# Plot output characteristics
plt.figure(figsize=(10, 6))
colors = cm.viridis(np.linspace(0, 1, len(Vgs_values)))

for i, Vgs in enumerate(Vgs_values):
    Ids = [gfet.drain_current(Vgs, Vds) * 1e6 for Vds in Vds_range]  # Convert to µA
    plt.plot(Vds_range, Ids, label=f'Vgs = {Vgs:.2f} V', color=colors[i], linewidth=2)

plt.xlabel('Drain-Source Voltage Vds (V)', fontsize=12)
plt.ylabel('Drain Current Ids (µA)', fontsize=12)
plt.title('GFET Output Characteristics', fontsize=14, fontweight='bold')
plt.legend(loc='best')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## Transfer Characteristics (Ids vs Vgs)

In [ ]:
# Sweep parameters
Vgs_range = np.linspace(-2.0, 2.0, 200)
Vds_values = [0.1, 0.5, 1.0]

# Plot transfer characteristics
plt.figure(figsize=(10, 6))

for Vds in Vds_values:
    Ids = [abs(gfet.drain_current(Vgs, Vds)) * 1e6 for Vgs in Vgs_range]  # Convert to µA
    plt.semilogy(Vgs_range, Ids, label=f'Vds = {Vds:.1f} V', linewidth=2)

plt.xlabel('Gate-Source Voltage Vgs (V)', fontsize=12)
plt.ylabel('Drain Current |Ids| (µA)', fontsize=12)
plt.title('GFET Transfer Characteristics (Log Scale)', fontsize=14, fontweight='bold')
plt.legend(loc='best')
plt.grid(True, alpha=0.3, which='both')
plt.axvline(x=0, color='red', linestyle='--', alpha=0.5, label='Dirac Point')
plt.tight_layout()
plt.show()

## Ambipolar Conduction

Graphene FETs exhibit ambipolar behavior - conduction by both electrons and holes.

In [ ]:
# Plot ambipolar characteristics
Vgs_range = np.linspace(-2.0, 2.0, 200)
Vds = 1.0

Ids = [gfet.drain_current(Vgs, Vds) * 1e6 for Vgs in Vgs_range]
conductance = np.gradient(Ids, Vgs_range[1] - Vgs_range[0])

fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(10, 10))

# Current
ax1.plot(Vgs_range, Ids, linewidth=2, color='blue')
ax1.axvline(x=0, color='red', linestyle='--', alpha=0.5)
ax1.set_xlabel('Gate-Source Voltage Vgs (V)', fontsize=12)
ax1.set_ylabel('Drain Current Ids (µA)', fontsize=12)
ax1.set_title('Ambipolar Conduction', fontsize=14, fontweight='bold')
ax1.grid(True, alpha=0.3)
ax1.text(-1.5, max(Ids)*0.8, 'Hole\nConduction', fontsize=10, ha='center')
ax1.text(1.5, max(Ids)*0.8, 'Electron\nConduction', fontsize=10, ha='center')

# Transconductance
ax2.plot(Vgs_range, conductance, linewidth=2, color='green')
ax2.axvline(x=0, color='red', linestyle='--', alpha=0.5)
ax2.axhline(y=0, color='black', linestyle='-', alpha=0.3)
ax2.set_xlabel('Gate-Source Voltage Vgs (V)', fontsize=12)
ax2.set_ylabel('Transconductance gm (µS)', fontsize=12)
ax2.set_title('Transconductance vs Gate Voltage', fontsize=14, fontweight='bold')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## Parameter Sensitivity Analysis

In [ ]:
# Analyze mobility effect
Vgs_range = np.linspace(-2.0, 2.0, 100)
Vds = 1.0
mobility_values = [5000, 10000, 20000, 40000]  # cm^2/V·s

plt.figure(figsize=(10, 6))

for mu in mobility_values:
    gfet_temp = AnalyticalGFET(mu=mu)
    Ids = [abs(gfet_temp.drain_current(Vgs, Vds)) * 1e6 for Vgs in Vgs_range]
    plt.semilogy(Vgs_range, Ids, label=f'µ = {mu} cm²/V·s', linewidth=2)

plt.xlabel('Gate-Source Voltage Vgs (V)', fontsize=12)
plt.ylabel('Drain Current |Ids| (µA)', fontsize=12)
plt.title('Effect of Mobility on GFET Characteristics', fontsize=14, fontweight='bold')
plt.legend(loc='best')
plt.grid(True, alpha=0.3, which='both')
plt.tight_layout()
plt.show()

print("Higher mobility leads to:")
print("  - Higher drain current")
print("  - Better on/off ratio")
print("  - Improved device performance")

## Summary

Key observations:
- Ambipolar conduction with Dirac point at Vgs ≈ 0V
- Carrier density modulation by gate voltage
- Mobility strongly affects device performance
- Current saturation at high Vds